In [3]:
! pip install -r requirements.txt --quiet

In [ ]:
from dotenv import load_dotenv
from os import environ
from langchain_openai import AzureChatOpenAI
from utils import pretty_print_response
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent


load_dotenv(override=True)

True

In [2]:
llm = AzureChatOpenAI(
        temperature=0,
        azure_deployment=environ["AZURE_OPENAI_MODEL"],
        api_version=environ["AZURE_OPENAI_API_VERSION"]
    )

In [ ]:
async with MultiServerMCPClient(
{
    "weather": {
        "url": f"{environ["MCP_WEATHER_SERVER_URL"]}/sse",
        "transport": "sse",
    }
}) as client:
    agent = create_react_agent(llm, client.get_tools())
    reponse = await agent.ainvoke({"messages": "what is the weather in sf"})
    pretty_print_response(reponse)


Human: what is the weather in sf


AI: (Tool Call Triggered)
   Calling `get_weather_tool` with args: {"location":"sf"}


Tool `get_weather_tool` Response: It's 60 degrees and foggy.


AI: The weather in San Francisco is currently 60°F and foggy.




In [4]:

reponse

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='632df805-4dbf-4731-866c-cbc40d3209bf'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_83zgSqOq0634VQNclTbNbAKU', 'function': {'arguments': '{"location":"sf"}', 'name': 'get_weather_tool'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 114, 'total_tokens': 130, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BPfmVfzMzNFOl0bshjkHUwbGTbTeJ', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': Fa